# Hyperparameter selection with Ray Tune

In [1]:
import os
from datetime import datetime
import time
import random
import warnings
import joblib
import warnings
import gc
# My modules
from config import Config
from logger import init_logger
from common_utils import set_seeds, read_csvs, stratify_split, setup, get_data_dfs, get_loaders
from model import Model
from train_loop_functions import train_epoch, valid_epoch

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score

import torch
from torch import nn
from torch.cuda.amp import GradScaler

# hyperparameter tuning
from functools import partial
import ray
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
from ray.tune.suggest.ax import AxSearch
from ray.tune.suggest.bayesopt import BayesOptSearch

In [2]:
%load_ext autoreload
%autoreload 2

# Setup

In [3]:
set_seeds(Config.seed)
LOGGER = init_logger() # uses Python's logging framework

# Tuning

In [4]:
# all params are passed in by Tune. this is the "objective" function.
def train_main(config, checkpoint_dir=None, data_dir=None):
    assert config is not None
    # -------- DATASETS AND LOADERS --------
    data_df, _, _ = read_csvs(data_dir, Config.debug, num_samples=7000)
    train_folds = stratify_split(data_df, Config.fold_num, Config.seed, Config.target_col)
    
    # select only one of the folds (fold 0)
    train_df, valid_df = get_data_dfs(train_folds, 0)
    train_dataloader, valid_dataloader = get_loaders(train_df, valid_df,
                                                     config["batch_size"], 
                                                     data_dir+'/train_images')
    
    # -------- MODEL --------
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model, optimizer, scheduler, criterion = setup(Config.model_arch, 
                                                   config["lr"], 
                                                   config["is_amsgrad"], 
                                                   num_labels=data_df.label.nunique(), 
                                                   fc_layer={
                                                       'middle_fc': config["middle_fc"], 
                                                        'middle_fc_size': config["middle_fc_size"]
                                                    }, 
                                                   weight_decay=config["weight_decay"],
                                                   device=device,
                                                   checkpoint=None)
    
    # The `checkpoint_dir` parameter gets passed by Ray Tune when a checkpoint should be restored.
    if checkpoint_dir:
        checkpoint = os.path.join(checkpoint_dir, "checkpoint")
        model_state, optimizer_state = torch.load(checkpoint)
        model.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)
        
    # EPOCHS TRAIN
    for e in range(10):
        # TRAIN
        avg_training_loss = train_epoch(train_dataloader, model, 
                                      criterion, optimizer, 
                                      scheduler, GradScaler(), 
                                      config["accum_iter"], LOGGER,
                                      device)

        # VALIDATE
        avg_validation_loss, preds = valid_epoch(valid_dataloader, model, criterion, LOGGER, device)
  
        validation_labels = valid_df[Config.target_col].values
        accuracy = accuracy_score(y_true=validation_labels, y_pred=preds)
        
        # SAVE CHECKPOINT.
        # It is automatically registered with Ray Tune and will potentially
        # be passed as the `checkpoint_dir` parameter in future iterations.
        with tune.checkpoint_dir(step=e) as checkpoint_dir:
            path = os.path.join(checkpoint_dir, "checkpoint")
            torch.save((model.state_dict(), optimizer.state_dict()), path)

        tune.report(loss=avg_validation_loss, accuracy=accuracy)
        gc.collect()

    print("Finished Training")

In [5]:
def main(num_samples=50, max_num_epochs=10, gpus_per_trial=1):
    data_dir = os.path.abspath('./data')
    
    hyperconfig = {
        "is_amsgrad": False,
        "accum_iter": tune.choice([4,8]),
        "lr": tune.loguniform(1e-3, 1e-2),
        "batch_size": tune.choice([8, 16]),
        "weight_decay": tune.choice([1/8 * 0.001, 1/2 * 0.001, 0.]),
        # try adding a FC layer to the classifier portion of the model
        "middle_fc": False,
        "middle_fc_size": tune.choice([512, 256])
    }
    
    scheduler = ASHAScheduler(
        #metric="loss",
        mode="min",
        max_t=max_num_epochs,
        grace_period=2,
        reduction_factor=2)
    reporter = CLIReporter(metric_columns=["loss", "accuracy", "training_iteration"])
    
    search_alg = AxSearch(metric="loss", mode="min")
    #search_alg = BayesOptSearch(metric="loss", mode="min")
    result = tune.run(
        partial(train_main, data_dir=data_dir),
        name="ax2",
        metric="loss",
        resources_per_trial={"cpu": os.cpu_count(), "gpu": gpus_per_trial},
        config=hyperconfig,
        num_samples=num_samples,
        scheduler=scheduler,
        progress_reporter=reporter,
        search_alg=search_alg,
        local_dir='./ray-results',
        checkpoint_score_attr='min-loss'
    )

    best_trial = result.get_best_trial("loss", "min", "last")
    print("Best trial config: {}".format(best_trial.config))
    print("Best trial final validation loss: {}".format(
        best_trial.last_result["loss"]))
    print("Best trial final validation accuracy: {}".format(
        best_trial.last_result["accuracy"]))

In [ ]:
main()

2020-12-26 22:04:11,463	INFO services.py:1092 -- View the Ray dashboard at http://127.0.0.1:8265
2020-12-26 22:04:12,128	WARNING experiment.py:274 -- No name detected on trainable. Using DEFAULT.
2020-12-26 22:04:12,129	INFO registry.py:65 -- Detected unknown callable for trainable. Converting to class.
[INFO 12-26 22:04:12] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 2 decimal points.
[INFO 12-26 22:04:12] ax.service.utils.instantiation: Inferred value type of ParameterType.INT for parameter accum_iter. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 12-26 22:04:12] ax.service.utils.instantiation: Inferred value type of ParameterType.INT for parameter batch_size. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'b

== Status ==
Memory usage on this node: 5.3/31.3 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None
Resources requested: 8/8 CPUs, 1/1 GPUs, 0.0/17.53 GiB heap, 0.0/6.05 GiB objects (0/1.0 accelerator_type:RTX)
Result logdir: /opt/favordata/AI/Felix/kaggle-cassava/ray-results/ax2
Number of trials: 1/50 (1 RUNNING)
+------------------+----------+-------+--------------+--------------+--------------+------------+-------------+------------------+----------------+
| Trial name       | status   | loc   |   accum_iter |   batch_size | is_amsgrad   |         lr | middle_fc   |   middle_fc_size |   weight_decay |
|------------------+----------+-------+--------------+--------------+--------------+------------+-------------+------------------+----------------|
| DEFAULT_492b1aee | RUNNING  |       |            8 |           16 | False        | 0.00143511 | False       |              256 |              0 |
+------------------+----------+-------+

Result for DEFAULT_492b1aee:
  accuracy: 0.696
  date: 2020-12-26_22-20-53
  done: false
  experiment_id: 026f4fb8ae0f433b879dee9624236d14
  experiment_tag: 1_accum_iter=8,batch_size=16,is_amsgrad=False,lr=0.0014351,middle_fc=False,middle_fc_size=256,weight_decay=0.0
  hostname: Nevsky
  iterations_since_restore: 5
  loss: 1.2652046593752775
  node_ip: 10.0.0.200
  pid: 14605
  should_checkpoint: true
  time_since_restore: 999.8098523616791
  time_this_iter_s: 199.54348397254944
  time_total_s: 999.8098523616791
  timestamp: 1609021253
  timesteps_since_restore: 0
  training_iteration: 5
  trial_id: 492b1aee
  
== Status ==
Memory usage on this node: 7.6/31.3 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: -1.2600499976765025 | Iter 2.000: -1.252114293792031
Resources requested: 8/8 CPUs, 1/1 GPUs, 0.0/17.53 GiB heap, 0.0/6.05 GiB objects (0/1.0 accelerator_type:RTX)
Result logdir: /opt/favordata/AI/Felix/kaggle-cassava/ray-results/ax2
Number of trials: 

Result for DEFAULT_492b1aee:
  accuracy: 0.6937142857142857
  date: 2020-12-26_22-34-11
  done: false
  experiment_id: 026f4fb8ae0f433b879dee9624236d14
  experiment_tag: 1_accum_iter=8,batch_size=16,is_amsgrad=False,lr=0.0014351,middle_fc=False,middle_fc_size=256,weight_decay=0.0
  hostname: Nevsky
  iterations_since_restore: 9
  loss: 1.2720156517895784
  node_ip: 10.0.0.200
  pid: 14605
  should_checkpoint: true
  time_since_restore: 1797.850442647934
  time_this_iter_s: 199.55387783050537
  time_total_s: 1797.850442647934
  timestamp: 1609022051
  timesteps_since_restore: 0
  training_iteration: 9
  trial_id: 492b1aee
  
== Status ==
Memory usage on this node: 7.6/31.3 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: -1.25368201082403 | Iter 4.000: -1.2600499976765025 | Iter 2.000: -1.252114293792031
Resources requested: 8/8 CPUs, 1/1 GPUs, 0.0/17.53 GiB heap, 0.0/6.05 GiB objects (0/1.0 accelerator_type:RTX)
Result logdir: /opt/favordata/AI/Felix/kaggle-cassava/ray-resu

[INFO 12-26 22:37:30] ax.service.ax_client: Completed trial 0 with data: {'loss': (1.27, 0.0)}.
[INFO 12-26 22:37:30] ax.service.ax_client: Generated new trial 2 with parameters {'lr': 0.0, 'accum_iter': 4, 'batch_size': 8, 'weight_decay': 0.0, 'middle_fc_size': 512, 'is_amsgrad': False, 'middle_fc': False}.


...

(pid=10348) Epoch     0: adjusting learning rate of group 0 to 8.0861e-03.
Result for DEFAULT_5135a650:
  accuracy: 0.11028571428571429
  date: 2020-12-27_02-51-22
  done: false
  experiment_id: 116aa590ae1a418ab7886c7c3df07ef8
  experiment_tag: 16_accum_iter=4,batch_size=16,is_amsgrad=False,lr=0.0080861,middle_fc=False,middle_fc_size=256,weight_decay=0.0005
  hostname: Nevsky
  iterations_since_restore: 1
  loss: 1.6356067332354458
  node_ip: 10.0.0.200
  pid: 10348
  should_checkpoint: true
  time_since_restore: 201.60954880714417
  time_this_iter_s: 201.60954880714417
  time_total_s: 201.60954880714417
  timestamp: 1609037482
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 5135a650
  
== Status ==
Memory usage on this node: 7.4/31.3 GiB
Using AsyncHyperBand: num_stopped=15
Bracket: Iter 8.000: -1.1740320362827994 | Iter 4.000: -1.2448209036480296 | Iter 2.000: -1.5774462288076228
Resources requested: 8/8 CPUs, 1/1 GPUs, 0.0/17.53 GiB heap, 0.0/6.05 GiB objects (0/1.

[INFO 12-27 02:54:41] ax.service.ax_client: Completed trial 15 with data: {'loss': (1.63, 0.0)}.


Result for DEFAULT_5135a650:
  accuracy: 0.11314285714285714
  date: 2020-12-27_02-54-41
  done: true
  experiment_id: 116aa590ae1a418ab7886c7c3df07ef8
  experiment_tag: 16_accum_iter=4,batch_size=16,is_amsgrad=False,lr=0.0080861,middle_fc=False,middle_fc_size=256,weight_decay=0.0005
  hostname: Nevsky
  iterations_since_restore: 2
  loss: 1.6320970340208574
  node_ip: 10.0.0.200
  pid: 10348
  should_checkpoint: true
  time_since_restore: 400.20594358444214
  time_this_iter_s: 198.59639477729797
  time_total_s: 400.20594358444214
  timestamp: 1609037681
  timesteps_since_restore: 0
  training_iteration: 2
  trial_id: 5135a650
  
== Status ==
Memory usage on this node: 7.4/31.3 GiB
Using AsyncHyperBand: num_stopped=16
Bracket: Iter 8.000: -1.1740320362827994 | Iter 4.000: -1.2448209036480296 | Iter 2.000: -1.5865291866389188
Resources requested: 8/8 CPUs, 1/1 GPUs, 0.0/17.53 GiB heap, 0.0/6.05 GiB objects (0/1.0 accelerator_type:RTX)
Result logdir: /opt/favordata/AI/Felix/kaggle-cassav

[INFO 12-27 02:54:41] ax.service.ax_client: Generated new trial 17 with parameters {'lr': 0.01, 'accum_iter': 8, 'batch_size': 8, 'weight_decay': 0.0, 'middle_fc_size': 512, 'is_amsgrad': False, 'middle_fc': False}.


(pid=12749) Epoch     0: adjusting learning rate of group 0 to 2.5192e-03.
